Initial Attempt at  Machine Learning Python Project with FRED


First install pandas, requests (for getting data from FRED),  scikit,( is for machine learning apps) and matplotlib ( for plotting and graphing) seaborn is also for graphing.
Also dotenv libraries

In [48]:
!pip install pandas requests scikit-learn matplotlib seaborn
!pip install python-dotenv
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 9.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 15.8 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.8-cp312-cp312-macosx_11_0_arm64.whl size=264225 sha256=5593b5bbbce78e3d118812a7a276d1593d1818185f768b0e0851e0863048be4b
  Stored in directory: /Users/Chris/Library/Caches/pip/wheels/8f/65/34/456800445efeafb05164fe95285c70e81ba1d96bae30f43917
Successfully built peewee


This next section is to load the dotenv and the get_fred_data function

In [22]:
from dotenv import load_dotenv
import os
import requests
load_dotenv()

fred_api_key = os.getenv('FRED_API_KEY') 



   

In [35]:
from lxml import etree 
import pandas as pd
from json import JSONDecodeError
import requests
from dotenv import load_dotenv
import os

def get_fred_data(series_id, start_date, end_date):
    """
    Fetches economic data for a given FRED series ID within a specified date range.

    Args:
        series_id (str): The FRED series ID (e.g., 'UNRATE' for unemployment rate).
        start_date (str): The start date in YYYY-MM-DD format (e.g., '2020-01-01').
        end_date (str): The end date in YYYY-MM-DD format (e.g., '2024-12-31').

    Returns:
        pandas.DataFrame: A DataFrame containing the economic data, indexed by date.

    Raises:
        ValueError: If the FRED API key is not found in the environment, 
                    the response status code is not 200, 
                    or the API response cannot be parsed. 
    """

    load_dotenv()
    fred_api_key = os.getenv('FRED_API_KEY')

    if not fred_api_key:
        raise ValueError("FRED API key not found in the environment. Please set the 'FRED_API_KEY' environment variable.")

    url = f"https://api.stlouisfed.org/fred/series/observations?" \
          f"series_id={series_id}&observation_start={start_date}&observation_end={end_date}" \
          f"&api_key={fred_api_key}" \
          f"&file_type=xml"  # Specify XML output

    try:
        response = requests.get(url)
        response.raise_for_status() 

        root = etree.fromstring(response.content) 
        data = []
        for observation in root.findall('observation'):
            date = observation.get('date')
            value = float(observation.get('value'))
            data.append({'date': date, 'value': value})

        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')
        return df

    except requests.exceptions.RequestException as e:
        raise ValueError(f"Failed to retrieve data from FRED API: {e}")
    except Exception as e:  # Catch general exceptions
        raise ValueError(f"An error occurred while processing the FRED API response: {e}")



In [44]:
# Example usage prints the first 12 lines:
try:
    unemployment_rate = get_fred_data('FEDFUNDS', '2010-01-01', '2023-12-31')
    print(unemployment_rate.head(12))  # Display the first few rows of the DataFrame
except ValueError as e:
    print(f"Error: {e}")

            value
date             
2010-01-01   0.11
2010-02-01   0.13
2010-03-01   0.16
2010-04-01   0.20
2010-05-01   0.20
2010-06-01   0.18
2010-07-01   0.18
2010-08-01   0.19
2010-09-01   0.19
2010-10-01   0.19
2010-11-01   0.19
2010-12-01   0.18


In [49]:
import yfinance as yf

# Download stock data (e.g., S&P 500)
sp500 = yf.download('^GSPC')
print (sp500.head())

[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open Volume
Ticker          ^GSPC      ^GSPC      ^GSPC      ^GSPC  ^GSPC
Date                                                         
1927-12-30  17.660000  17.660000  17.660000  17.660000      0
1928-01-03  17.760000  17.760000  17.760000  17.760000      0
1928-01-04  17.719999  17.719999  17.719999  17.719999      0
1928-01-05  17.549999  17.549999  17.549999  17.549999      0
1928-01-06  17.660000  17.660000  17.660000  17.660000      0
